In [ ]:
%cd ./ProphecyPlus
!pwd
!ls -lt

/content/ProphecyPlus
/content/ProphecyPlus
total 28
drwxr-xr-x  5 root root 4096 Apr 12 20:47  prophecy
-rw-r--r--  1 root root  172 Apr 12 20:47  requirements.txt
drwxr-xr-x  3 root root 4096 Apr 12 20:47  examples
drwxr-xr-x 10 root root 4096 Apr 12 20:47  dataset_models
-rw-r--r--  1 root root 5775 Apr 12 20:47  README.md
drwxr-xr-x  7 root root 4096 Apr 12 20:47 'Legacy Code'


In [ ]:
!pip install -r requirements.txt

**Obtain fingerprints array for CLIP (model name: , layer: )**

**Contains inputs for truck and car.**

**Zero-shot classification of CLIP is used to classify trucks and cars.**

**The labels file contains correct vs incorrect zero-shot classifications. Inputs for which the zero-class output matches GT label are considered correct else incorrect**


**Access the fingerprints and labels files.**

In [ ]:
import requests

# Dropbox shared link (ensure it's the correct URL)
shared_link = "https://www.dropbox.com/scl/fi/mz91a1xqqcmcfhned53hi/CLIP_data.zip?rlkey=iuwz8cab7tmuf6o563vywms73&st=ad36xboe&dl=0"

# Modify the link for direct download
direct_download_url = shared_link.replace("dl=0", "dl=1")

# Filepath to save the downloaded file
file_path = "CLIP_data.zip"

try:
    # Send a GET request to download the file
    response = requests.get(direct_download_url)
    response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)

    # Save the content to a local file
    with open(file_path, "wb") as file:
        file.write(response.content)
    print(f"File downloaded successfully and saved as {file_path}")

except requests.exceptions.RequestException as e:
    print(f"Error occurred: {e}")



File downloaded successfully and saved as CLIP_data.zip


In [ ]:
!unzip CLIP_data.zip

Archive:  CLIP_data.zip
   creating: data/
  inflating: data/truck_car_fingerprints.csv  
  inflating: data/truck_car_labels.csv  
  inflating: data/truck_car_labels_new.csv  


In [ ]:
import numpy as np
data = np.genfromtxt('data/truck_car_fingerprints.csv', delimiter=',')
labels = np.genfromtxt('data/truck_car_labels.csv', delimiter=',')
labelsnew = np.genfromtxt('data/truck_car_labels_new.csv', delimiter=',')

print(np.shape(data))
print(np.shape(labels))
print(np.shape(labelsnew))


(497, 263168)
(497,)
(497,)


**labels:**

1: truck or car correctly classified

0: truck or car mis-classified.

In [ ]:
labels_MIS = np.where(labels == 0)[0]
labels_COR = np.where(labels == 1)[0]

data_0_MIS =[]
data_1_COR = []


lbl_0_MIS = []
lbl_1_COR = []


for indx in range(0,len(data)):
  if indx in labels_MIS:
    data_0_MIS.append(data[indx])
    lbl_0_MIS.append(labels[indx])
  elif indx in labels_COR:
    data_1_COR.append(data[indx])
    lbl_1_COR.append(labels[indx])
  else:
    print("wrong")

print("Misclass:", np.shape(data_0_MIS))
print("Correct:", np.shape(data_1_COR))


print(np.shape(lbl_0_MIS))
print(np.shape(lbl_1_COR))



data_train_2lbls = []
lbl_train_2lbls = []
data_test_2lbls = []
lbl_test_2lbls = []
for indx in range(0,len(data_0_MIS)):
  if (indx < len(data_0_MIS)/2):
    data_test_2lbls.append(data_0_MIS[indx])
    lbl_test_2lbls.append(lbl_0_MIS[indx])
  else:
    data_train_2lbls.append(data_0_MIS[indx])
    lbl_train_2lbls.append(lbl_0_MIS[indx])

for indx in range(0,len(data_1_COR)):
  if (indx < len(data_1_COR)/4):
    data_test_2lbls.append(data_1_COR[indx])
    lbl_test_2lbls.append(lbl_1_COR[indx])
  else:
    data_train_2lbls.append(data_1_COR[indx])
    lbl_train_2lbls.append(lbl_1_COR[indx])

print("TRAIN:")
print(np.shape(data_train_2lbls))
print(np.shape(lbl_train_2lbls))
print("TEST:")
print(np.shape(data_test_2lbls))
print(np.shape(lbl_test_2lbls))

Misclass: (97, 263168)
Correct: (400, 263168)
(97,)
(400,)
TRAIN:
(348, 263168)
(348,)
TEST:
(149, 263168)
(149,)


**labels_new:**

Correctly classified car - label 1

Misclassified car - label 0

Correctly classified truck - label 2

Misclassified truck - label 3.

In [ ]:
labels_0 = np.where(labelsnew == 0)[0]
labels_1 = np.where(labelsnew == 1)[0]
labels_2 = np.where(labelsnew == 2)[0]
labels_3 = np.where(labelsnew == 3)[0]

data_0 =[]
data_1 = []
data_2 =[]
data_3 = []

lbl_0 = []
lbl_1 = []
lbl_2 = []
lbl_3 = []

for indx in range(0,len(data)):
  if indx in labels_0:
    data_0.append(data[indx])
    lbl_0.append(labelsnew[indx])
  elif indx in labels_1:
    data_1.append(data[indx])
    lbl_1.append(labelsnew[indx])
  elif indx in labels_2:
    data_2.append(data[indx])
    lbl_2.append(labelsnew[indx])
  elif indx in labels_3:
    data_3.append(data[indx])
    lbl_3.append(labelsnew[indx])
  else:
    print("wrong")

print("CAR MIS:",np.shape(data_0))
print("CAR COR:",np.shape(data_1))
print("TRUCK COR:",np.shape(data_2))
print("TRUCK MIS:",np.shape(data_3))

print("CAR MIS:",np.shape(lbl_0))
print("CAR COR:", np.shape(lbl_1))
print("TRUCK COR:", np.shape(lbl_2))
print("TRUCK MIS:", np.shape(lbl_3))


data_train = []
lbl_train = []
data_test = []
lbl_test = []

data_0_cnt = 0
data_1_cnt = 0
data_2_cnt = 0
data_3_cnt = 0

for indx in range(0,len(data_0)):
  if (indx < len(data_0)/2):
    data_0_cnt += 1
    data_test.append(data_0[indx])
    lbl_test.append(lbl_0[indx])
  else:
    data_train.append(data_0[indx])
    lbl_train.append(lbl_0[indx])

for indx in range(0,len(data_1)):
  if (indx < len(data_1)/4):
    data_1_cnt += 1
    data_test.append(data_1[indx])
    lbl_test.append(lbl_1[indx])
  else:
    data_train.append(data_1[indx])
    lbl_train.append(lbl_1[indx])

for indx in range(0,len(data_3)):
  if (indx < len(data_3)/2):
    data_3_cnt += 1
    data_test.append(data_3[indx])
    lbl_test.append(lbl_3[indx])
  else:
    data_train.append(data_3[indx])
    lbl_train.append(lbl_3[indx])

for indx in range(0,len(data_2)):
  if (indx < len(data_2)/4):
    data_2_cnt += 1
    data_test.append(data_2[indx])
    lbl_test.append(lbl_2[indx])
  else:
    data_train.append(data_2[indx])
    lbl_train.append(lbl_2[indx])

print(np.shape(data_train))
print(np.shape(lbl_train))
print(np.shape(data_test))
print(np.shape(lbl_test))


print(data_0_cnt)
print(data_1_cnt)
print(data_2_cnt)
print(data_3_cnt)

CAR MIS: (41, 263168)
CAR COR: (200, 263168)
TRUCK COR: (200, 263168)
TRUCK MIS: (56, 263168)
CAR MIS: (41,)
CAR COR: (200,)
TRUCK COR: (200,)
TRUCK MIS: (56,)
(348, 263168)
(348,)
(149, 263168)
(149,)
21
50
50
28


In [ ]:
# Save the array as an NPY file
np.save('./features_train.npy', data_train)
np.save('./cor_inc_train.npy', lbl_train)
np.save('./features_test.npy', data_test)
np.save('./cor_inc_test.npy', lbl_test)

In [ ]:
np.save('./features_train_2lbls.npy', data_train_2lbls)
np.save('./cor_inc_train_2lbls.npy', lbl_train_2lbls)
np.save('./features_test_2lbls.npy', data_test_2lbls)
np.save('./cor_inc_test_2lbls.npy', lbl_test_2lbls)

**INVOKE PROPHECY**

In [ ]:
!ls -lt

total 2043740
-rw-r--r--  1 root root      1320 Apr 12 20:51  cor_inc_test_2lbls.npy
-rw-r--r--  1 root root 313696384 Apr 12 20:51  features_test_2lbls.npy
-rw-r--r--  1 root root      2912 Apr 12 20:51  cor_inc_train_2lbls.npy
-rw-r--r--  1 root root 732659840 Apr 12 20:51  features_train_2lbls.npy
-rw-r--r--  1 root root      1320 Apr 12 20:51  cor_inc_test.npy
-rw-r--r--  1 root root 313696384 Apr 12 20:51  features_test.npy
-rw-r--r--  1 root root      2912 Apr 12 20:51  cor_inc_train.npy
-rw-r--r--  1 root root 732659840 Apr 12 20:51  features_train.npy
drwxr-xr-x  5 root root      4096 Apr 12 20:47  prophecy
-rw-r--r--  1 root root       172 Apr 12 20:47  requirements.txt
drwxr-xr-x  3 root root      4096 Apr 12 20:47  examples
drwxr-xr-x 10 root root      4096 Apr 12 20:47  dataset_models
-rw-r--r--  1 root root      5775 Apr 12 20:47  README.md
drwxr-xr-x  7 root root      4096 Apr 12 20:47 'Legacy Code'


In [ ]:
import os
import shutil

if os.path.exists('./results/CLIP/rules/'):
  shutil.rmtree('./results/CLIP/rules/')

os.makedirs('./results/CLIP/rules/')

!pwd

import time

start_time = time.time()

!python -m prophecy.main -wd './results/CLIP/rules/' analyze -tx './features_train.npy' -ty './cor_inc_train.npy' -vx './features_test.npy' -vy './cor_inc_test.npy' -type 3 -inptype 1

print(" ELAPSED TIME:", time.time() - start_time)

/content/ProphecyPlus
Instructions for updating:
non-resource variables are not supported in the long term
/opt/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
CONFIG PARAMS:
TYPE: 3 ,INP TYPE: 1 ,ACTS: False ,Top: False
Invoking Dec-tree classifier based on FEATURES
Inputs: (neuron signature (On/Off activations) dataset)(labels dataset)
(348, 263168) (348,)

RULES FROM LAYER CURRENT IN TERMS OF FEATURES

Obtained all paths
Processing paths for validation set: 100% 29/29 [00:00<00:00, 492448.65it/s]
InV 0
PRINTING ALL RULES.
 ELAPSED TIME: 75.16183710098267


In [ ]:
import pandas as pd

_output_path = "./results/CLIP/rules/ruleset.csv"


print("****** RULES ********")
df_op = pd.read_csv(_output_path)

print("RULES FOR CORRECT CLASS CAR:")
df_op_cor_car = df_op[df_op['label'] == 1]
df_op_cor_car.to_csv('./results/CLIP/rules/ruleset_cor_car.csv')

print("RULES FOR CORRECT CLASS TRUCK:")
df_op_cor_truck = df_op[df_op['label'] == 2]
df_op_cor_truck.to_csv('./results/CLIP/rules/ruleset_cor_truck.csv')

print("RULES FOR MIS CLASS CAR:")
df_op_cor_car = df_op[df_op['label'] == 0]
df_op_cor_car.to_csv('./results/CLIP/rules/ruleset_mis_car.csv')

print("RULES FOR MISCLASS TRUCK:")
df_op_cor_truck = df_op[df_op['label'] == 3]
df_op_cor_truck.to_csv('./results/CLIP/rules/ruleset_mis_truck.csv')

****** RULES ********
RULES FOR CORRECT CLASS CAR:
RULES FOR CORRECT CLASS TRUCK:
RULES FOR MIS CLASS CAR:
RULES FOR MISCLASS TRUCK:


In [ ]:
_output_path = "./results/CLIP/rules/ruleset_cor_car.csv"


print("****** TOP RULE FOR CORRECT CLASS CAR ********")
df_op = pd.read_csv(_output_path)
top = df_op[df_op.index == 0]
top


****** TOP RULE FOR CORRECT CLASS CAR ********


,Unnamed: 0,layer,layer_count,neurons,signature,support,label,train_coverage,train_precision,train_recall,train_f1,test_coverage,test_precision,test_recall,test_f1,f1,kind
0,0,current,1,"[396, 188122, 157846, 203801, 88270]","['<=', -2.9094423055648804, '<=', 6.0017716884...",114,1,32.758621,100,76.0,86.363636,29.530201,59.090909,52.0,55.319149,70.841393,correct


In [ ]:
_output_path = "./results/CLIP/rules/ruleset_cor_truck.csv"


print("****** TOP RULE FOR CORRECT CLASS TRUCK ********")
df_op = pd.read_csv(_output_path)
top = df_op[df_op.index == 0]
top

****** TOP RULE FOR CORRECT CLASS TRUCK ********


,Unnamed: 0,layer,layer_count,neurons,signature,support,label,train_coverage,train_precision,train_recall,train_f1,test_coverage,test_precision,test_recall,test_f1,f1,kind
0,8,current,1,"[396, 33, 36996, 154735, 126911, 92660]","['>', -2.9094423055648804, '<=', -0.3497829437...",117,2,33.62069,100,78.0,87.640449,30.872483,58.695652,54.0,56.25,71.945225,correct


In [ ]:
_output_path = "./results/CLIP/rules/ruleset_mis_truck.csv"


print("****** TOP RULE FOR MIS CLASS TRUCK ********")
df_op = pd.read_csv(_output_path)
top = df_op[df_op.index == 0]
top

****** TOP RULE FOR MIS CLASS TRUCK ********


,Unnamed: 0,layer,layer_count,neurons,signature,support,label,train_coverage,train_precision,train_recall,train_f1,test_coverage,test_precision,test_recall,test_f1,f1,kind
0,16,current,1,"[396, 33, 219348, 4638, 153077]","['>', -2.9094423055648804, '>', -0.34978294372...",10,3,2.873563,100,35.714286,52.631579,4.697987,42.857143,10.714286,17.142857,34.887218,correct


In [ ]:
_output_path = "./results/CLIP/rules/ruleset_mis_car.csv"


print("****** TOP RULE FOR MIS CLASS CAR ********")
df_op = pd.read_csv(_output_path)
df_op
#top = df_op[df_op.index == 0]
#top

****** TOP RULE FOR MIS CLASS CAR ********


,Unnamed: 0,layer,layer_count,neurons,signature,support,label,train_coverage,train_precision,train_recall,train_f1,test_coverage,test_precision,test_recall,test_f1,f1,kind
0,23,current,1,"[396, 33, 219348, 4638, 153077, 156943]","['>', -2.9094423055648804, '>', -0.34978294372...",8,0,2.298851,100,40.0,57.142857,0.000000,0.0,0.000000,0.000000,28.571429,correct
1,24,current,1,"[396, 188122, 157846, 203801, 239132]","['<=', -2.9094423055648804, '<=', 6.0017716884...",5,0,1.436782,100,25.0,40.000000,3.355705,20.0,4.761905,7.692308,23.846154,correct
2,25,current,1,"[396, 33, 36996, 154735, 212939, 9217, 239728]","['>', -2.9094423055648804, '<=', -0.3497829437...",3,0,0.862069,100,15.0,26.086957,0.671141,0.0,0.000000,0.000000,13.043478,correct
3,26,current,1,"[396, 33, 36996, 154735, 126911, 92660, 190638]","['>', -2.9094423055648804, '<=', -0.3497829437...",2,0,0.574713,100,10.0,18.181818,0.671141,100.0,4.761905,9.090909,13.636364,correct
4,27,current,1,"[396, 188122, 157846, 203801, 88270, 252104, 1...","['<=', -2.9094423055648804, '<=', 6.0017716884...",1,0,0.287356,100,5.0,9.523810,0.671141,0.0,0.000000,0.000000,4.761905,correct
5,28,current,1,"[396, 33, 219348, 12659, 150837]","['>', -2.9094423055648804, '>', -0.34978294372...",1,0,0.287356,100,5.0,9.523810,1.342282,50.0,4.761905,8.695652,9.109731,correct


**With 2 labels**

In [ ]:
import os
import shutil

if os.path.exists('./results/CLIP/rules/'):
  shutil.rmtree('./results/CLIP/rules/')

os.makedirs('./results/CLIP/rules/')

!pwd

import time

start_time = time.time()

!python -m prophecy.main -wd './results/CLIP/rules/' analyze -tx './features_train_2lbls.npy' -ty './cor_inc_train_2lbls.npy' -vx './features_test_2lbls.npy' -vy './cor_inc_test_2lbls.npy' -type 3 -inptype 1

print(" ELAPSED TIME in secs:", time.time() - start_time)

/content/ProphecyPlus
Instructions for updating:
non-resource variables are not supported in the long term
/opt/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
CONFIG PARAMS:
TYPE: 3 ,INP TYPE: 1 ,ACTS: False ,Top: False
Invoking Dec-tree classifier based on FEATURES
Inputs: (neuron signature (On/Off activations) dataset)(labels dataset)
(348, 263168) (348,)

RULES FROM LAYER CURRENT IN TERMS OF FEATURES

Obtained all paths
Processing paths for validation set: 100% 14/14 [00:00<00:00, 257544.98it/s]
InV 0
PRINTING ALL RULES.
 ELAPSED TIME in secs: 69.24363589286804


In [ ]:
import pandas as pd

_output_path = "./results/CLIP/rules/ruleset.csv"


print("****** RULES ********")
df_op = pd.read_csv(_output_path)

print("RULES FOR CORRECT CLASS:")
df_op_cor_car = df_op[df_op['label'] == 1]
df_op_cor_car.to_csv('./results/CLIP/rules/ruleset_cor.csv')



print("RULES FOR MIS CLASS:")
df_op_cor_car = df_op[df_op['label'] == 0]
df_op_cor_car.to_csv('./results/CLIP/rules/ruleset_mis.csv')



****** RULES ********
RULES FOR CORRECT CLASS:
RULES FOR MIS CLASS:


In [ ]:
_output_path = "./results/CLIP/rules/ruleset_cor.csv"


print("****** TOP RULE FOR CORRECT CLASS CAR ********")
df_op = pd.read_csv(_output_path)
top = df_op[df_op.index == 0]
top


****** TOP RULE FOR CORRECT CLASS CAR ********


,Unnamed: 0,layer,layer_count,neurons,signature,support,label,train_coverage,train_precision,train_recall,train_f1,test_coverage,test_precision,test_recall,test_f1,f1,kind
0,0,current,1,"[139445, 52507, 75351, 88245, 135012, 174394]","['>', -1.7880505919456482, '<=', 3.87554490566...",277,1,79.597701,100,92.333333,96.013865,69.127517,66.990291,69.0,67.980296,81.99708,correct


In [ ]:
_output_path = "./results/CLIP/rules/ruleset_mis.csv"


print("****** TOP RULE FOR CORRECT CLASS TRUCK ********")
df_op = pd.read_csv(_output_path)
top = df_op[df_op.index == 0]
top

****** TOP RULE FOR CORRECT CLASS TRUCK ********


,Unnamed: 0,layer,layer_count,neurons,signature,support,label,train_coverage,train_precision,train_recall,train_f1,test_coverage,test_precision,test_recall,test_f1,f1,kind
0,7,current,1,"[139445, 100732, 72927]","['<=', -1.7880505919456482, '>', 0.41531665623...",21,0,6.034483,100,43.75,60.869565,7.38255,36.363636,8.163265,13.333333,37.101449,correct
